In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.0


In [2]:
#loss function
def nll(scores_pred, eta):
    scores_neg = scores_pred[1]
    scores_pos = scores_pred[0]

    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

#pm.compile(optimizer='adam', loss=nll)

# Train and eval without partition

In [3]:
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=14505, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
         batch_size=1000,
         epochs=10, callbacks=[checkpoint])

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')
end = time.time()


Epoch 1/10
    273/Unknown - 9s 32ms/step - loss: 1497.9250
Epoch 00001: loss improved from inf to 1497.92505, saving model to ./chkpt1
273/273 [==============================] - 9s 32ms/step - loss: 1497.9250
Epoch 2/10
273/273 [==============================] - ETA: 0s - loss: 1216.5336
Epoch 00002: loss improved from 1497.92505 to 1216.53357, saving model to ./chkpt1
273/273 [==============================] - 9s 31ms/step - loss: 1216.5336
Epoch 3/10
273/273 [==============================] - ETA: 0s - loss: 1030.6790
Epoch 00003: loss improved from 1216.53357 to 1030.67896, saving model to ./chkpt1
273/273 [==============================] - 9s 31ms/step - loss: 1030.6790
Epoch 4/10
273/273 [==============================] - ETA: 0s - loss: 902.6467
Epoch 00004: loss improved from 1030.67896 to 902.64667, saving model to ./chkpt1
273/273 [==============================] - 9s 32ms/step - loss: 902.6467
Epoch 5/10
272/273 [============================>.] - ETA: 0s - loss: 810.1034
Epo

In [4]:
mr_score(ranks), mrr_score(ranks)

(437.08630485893417, 0.09560396082399875)

# Train with partition

In [5]:
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     max_ent_size=10000, 
                                     max_rel_size=237,
                                     scoring_type='TransE')

model.compile(optimizer='adam', loss=nll)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
         batch_size=1000, use_partitioning=True,
         epochs=10, callbacks=[checkpoint])

#start = time.time()
#ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')
#end = time.time()

Partitioning may take a while...
_split: memory before: 0.0Bytes, after: 13.001MB, consumed: 13.001MB; exec time: 230.68s
Epoch 1/10
    277/Unknown - 10s 38ms/step - loss: 1631.9161
Epoch 00001: loss improved from inf to 1631.91614, saving model to ./chkpt1
277/277 [==============================] - 11s 38ms/step - loss: 1631.9161
Epoch 2/10
276/277 [============================>.] - ETA: 0s - loss: 1542.3390
Epoch 00002: loss improved from 1631.91614 to 1541.34094, saving model to ./chkpt1
277/277 [==============================] - 8s 30ms/step - loss: 1541.3409
Epoch 3/10
277/277 [==============================] - ETA: 0s - loss: 1460.8374
Epoch 00003: loss improved from 1541.34094 to 1460.83740, saving model to ./chkpt1
277/277 [==============================] - 9s 34ms/step - loss: 1460.8374
Epoch 4/10
275/277 [============================>.] - ETA: 0s - loss: 1392.3140
Epoch 00004: loss improved from 1460.83740 to 1392.21301, saving model to ./chkpt1
277/277 [====================

In [6]:
#mr_score(ranks), mrr_score(ranks)